# Difftest Results

Connect to results database:

In [6]:
import db
from db import *
%run util.py

hostname = "cc1"
db.init(hostname)

'mysql://cc1:3306/project_b'

## Overview

In [7]:
import pandas as pd

session = db.make_session()

TABLE_NAMES = ["CLSmith", "CLSmith w. cldrive", "GitHub", "CLgen", "CLgen w. cl_launcher", "CLgen w. co"]
TABLES = [CLSmithResult, cldriveCLSmithResult, GitHubResult, CLgenResult, cl_launcherCLgenResult, coCLgenResult]

data = [
    ("#. Programs", [session.query(t.program_id).group_by(t.program_id).count() for t in TABLES]),
    ("#. Testbeds", [session.query(t.testbed_id).group_by(t.testbed_id).count() for t in TABLES]),
    ("#. Params", [session.query(t.params_id).group_by(t.params_id).count() for t in TABLES]),
    ("#. Results", [session.query(t).count() for t in TABLES]),
]
i, d = zip(*data)
overview = pd.DataFrame(list(d), index=i, columns=TABLE_NAMES)
overview

,CLSmith,CLSmith w. cldrive,GitHub,CLgen,CLgen w. cl_launcher,CLgen w. co
#. Programs,10001,10001,9238,3385,2385,3385
#. Testbeds,8,5,5,5,7,7
#. Params,4,4,4,4,4,4
#. Results,229616,200020,193477,49426,66780,53840


# Experimental Setup

### TestBeds

A testbed is a combination of host platform and OpenCL device.

In [8]:
import sqlalchemy as sql

q = session.query(Testbed).order_by(sql.func.field(Testbed.devtype, 'GPU', 'CPU', 'Emulator'))

data = []
for testbed in q:
    data.append(
        (testbed.id, [
            host_str(testbed.host), device_str(testbed.device),
            DRIVERS.get(testbed.driver, testbed.driver), testbed.opencl, testbed.devtype] +
         [session.query(t.testbed).filter(t.testbed == testbed).count() for t in TABLES]))
i, d = zip(*data)
testbeds = pd.DataFrame(list(d), index=i, columns=["Operating System", "Device", "Driver", "OpenCL", "Device type"] + [f"#. {t}" for t in TABLE_NAMES])

if len(CONFIGURATIONS) != session.query(Testbed).count():
    import sys
    print("warning: missing testbed(s)", file=sys.stderr)
testbeds

,Operating System,Device,Driver,OpenCL,Device type,#. CLSmith,#. CLSmith w. cldrive,#. GitHub,#. CLgen,#. CLgen w. cl_launcher,#. CLgen w. co
12,Ubuntu 16.04 64bit,Intel E5-2620 (pocl),0.14,2.0,3,320032,320032,295616,60528,76320,63104
15,CentOS 7.1 64bit,Intel Xeon Phi (?),1.2,1.2,ACCELERATOR,54408,0,0,0,0,42760
3,Ubuntu 16.04 64bit,NVIDIA GTX 1080,375.39,1.2,GPU,320032,320032,379576,45096,76320,61304
13,Ubuntu 16.04 64bit,Intel HD Haswell GT2,1.3,1.2,GPU,44312,0,0,108320,76320,0
9,Ubuntu 16.04 64bit,Intel E5-2620 v4,1.2.0.25,2.0,CPU,320032,320032,295616,108320,76320,95432
10,Ubuntu 16.04 64bit,Intel i5-4570,1.2.0.25,1.2,CPU,320032,320032,281392,0,76320,21968
14,CentOS 7.1 64bit,Intel E5-2650 v2,1.2.0.44,1.2,CPU,138048,0,0,0,76320,61288
11,Ubuntu 16.04 64bit,Oclgrind Simulator,16.10,1.2,Emulator,320032,320032,295616,73144,76320,84864


In [9]:
# push LaTex to Overleaf
!cd ~/docs/paper-project_b/ && git pull --rebase

import os
from collections import OrderedDict

def get_total_submitted(testbed: Testbed):
    submitable_results = [coCLgenResult, cl_launcherCLgenResult]
    
    def get_submitted(table):
        return session.query(table).filter(table.testbed_id == testbed.id, table.submitted).count()

    def get_generated(table):
        return session.query(table).filter(table.testbed_id == testbed.id, sql.or_(table.submitted, table.dupe)).count()
    
    return (
        sum(get_generated(table) for table in submitable_results), 
        sum(get_submitted(table) for table in submitable_results)
    )


def get_testbed_info(config_id, testbed_id):
    testbed = session.query(Testbed).filter(Testbed.id == testbed_id).first()
    d = OrderedDict()
    d["#."] = config_id
    d["Device"] = testbed.device
    d["Platform"] = platform_str(testbed.platform)
    d["Driver"] = driver_str(testbed.driver)
    d["OpenCL"] = testbed.opencl
    d["Operating system"] = host_str(testbed.host)
    d["Device type"] = devtype_str(testbed.devtype)
    d["B.R. Generated"], d["B.R. Submitted"] = get_total_submitted(testbed)
    return d

table = pd.DataFrame([get_testbed_info(*x) for x in CONFIGURATIONS])
with open(os.path.expanduser("~/docs/paper-project_b/build/tab/platforms.tex"), "w") as outfile:
    table.to_latex(buf=outfile, index=None)
!cd ~/docs/paper-project_b/build && git add . && git commit -m "auto: build/tab/platforms.tex" && git push
table

remote: Counting objects: 17, done
remote: Finding sources: 100% (12/12)
remote: Getting sizes: 100% (8/8)
remote: Compressing objects: 100% (399/399)
remote: Total 12 (delta 7), reused 9 (delta 6)
Unpacking objects: 100% (12/12), done.
From https://git.overleaf.com/8608915dsywxshwwjmw
   a717006..07b2c7c  master     -> origin/master
First, rewinding head to replay your work on top of it...
Fast-forwarded master to 07b2c7c8e37c1430e1b4471a5bd1a3b84189dde5.
On branch master
Your branch is up-to-date with 'origin/master'.
nothing to commit, working directory clean


,#.,Device,Platform,Driver,OpenCL,Operating system,Device type,B.R. Generated,B.R. Submitted
0,1,GeForce GTX 1080,NVIDIA CUDA,375.39,1.2,Ubuntu 16.04 64bit,GPU,13,7
1,2,Intel(R) HD Graphics Haswell GT2 Desktop,Intel Gen OCL Driver,1.3,1.2,Ubuntu 16.04 64bit,GPU,35,11
2,3,Intel(R) Xeon(R) CPU E5-2620 v4 @ 2.10GHz,Intel OpenCL,1.2.0.25,2.0,Ubuntu 16.04 64bit,CPU,10,5
3,4,Intel(R) Xeon(R) CPU E5-2650 v2 @ 2.60GHz,Intel OpenCL,1.2.0.44,1.2,CentOS 7.1 64bit,CPU,2,1
4,5,Intel(R) Core(TM) i5-4570 CPU @ 3.20GHz,Intel OpenCL,1.2.0.25,1.2,Ubuntu 16.04 64bit,CPU,4,4
5,6,Intel(R) Many Integrated Core Acceleration Card,Intel OpenCL,1.2,1.2,CentOS 7.1 64bit,Accelerator,0,0
6,7,pthread-Intel(R) Xeon(R) CPU E5-2620 v4 @ 2.10GHz,POCL,0.14,2.0,Ubuntu 16.04 64bit,CPU,170,52
7,8,Oclgrind Simulator,Oclgrind,16.10,1.2,Ubuntu 16.04 64bit,Emulator,0,0


In [10]:
%run mkmegatable.ipy

CLSmithResults GeForce GTX 1080 False {'w': 1, 'bf': 1132, 'c': 137, 'to': 1480, 'pass': 12754, 'fail': 0, 'total': '15504'}
CLgenResults GeForce GTX 1080 False {'w': 0, 'bf': 253, 'c': 1, 'to': 6, 'pass': 1178, 'fail': 0, 'total': '1438*'}


insufficient CLgenResults for GeForce GTX 1080 False


CLSmithResults GeForce GTX 1080 True {'w': 14, 'bf': 1135, 'c': 217, 'to': 1346, 'pass': 12858, 'fail': 0, 'total': '15570'}
CLgenResults GeForce GTX 1080 True {'w': 0, 'bf': 31, 'c': 0, 'to': 1, 'pass': 103, 'fail': 0, 'total': '135*'}


insufficient CLgenResults for GeForce GTX 1080 True


CLSmithResults Intel(R) HD Graphics Haswell GT2 Desktop False {'w': 0, 'bf': 185, 'c': 1062, 'to': 999, 'pass': 287, 'fail': 0, 'total': '2533*'}


...

CLSmithResults Oclgrind Simulator False {'w': 0, 'bf': 393, 'c': 5, 'to': 525, 'pass': 4472, 'fail': 0, 'total': '5395'}
CLgenResults Oclgrind Simulator False {'w': 0, 'bf': 0, 'c': 0, 'to': 0, 'pass': 0, 'fail': 0, 'total': '0*'}


insufficient CLgenResults for Oclgrind Simulator False


CLSmithResults Oclgrind Simulator True {'w': 3, 'bf': 405, 'c': 5, 'to': 520, 'pass': 4574, 'fail': 0, 'total': '5507'}
CLgenResults Oclgrind Simulator True {'w': 0, 'bf': 480, 'c': 0, 'to': 14, 'pass': 2224, 'fail': 0, 'total': '2718*'}


insufficient CLgenResults for Oclgrind Simulator True


Current branch master is up to date.
[master 00732ee] auto: build/tab/megatable.tex
 1 file changed, 7 insertions(+), 7 deletions(-)
Counting objects: 5, done.
Delta compression using up to 16 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 705 bytes | 0 bytes/s, done.
Total 5 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3)
remote: Updating references: 100% (1/1)
To https://git.overleaf.com/8608915dsywxshwwjmw
   07b2c7c..00732ee  master -> master


## Runtime Parameters

### cl_launcher

In [ ]:
CL_LAUNCHER_TABLE_NAMES = ["CLSmith", "CLgen w. cl_launcher"]
CL_LAUNCHER_TABLES = [CLSmithResult, cl_launcherCLgenResult]

q = session.query(cl_launcherParams).order_by(
        cl_launcherParams.gsize_x, cl_launcherParams.gsize_y, cl_launcherParams.gsize_z,
        cl_launcherParams.lsize_x, cl_launcherParams.lsize_y, cl_launcherParams.lsize_z,
        cl_launcherParams.optimizations)

data = []
for param in q:
    nresult_param = session.query(CLSmithResult).filter(CLSmithResult.params == param).count()
    data.append((
        param.id, [param.gsize, param.lsize, param.optimizations_on_off ] + [
            session.query(t).filter(t.params == param).count()
            for t in CL_LAUNCHER_TABLES
        ]))
i, d = zip(*data)

cl_launcher_params = pd.DataFrame(list(d), index=i, columns=[
    "Global size", "Local size", "Optimizations"] + [
        f"#. {t}" for t in CL_LAUNCHER_TABLE_NAMES])
cl_launcher_params

### cldrive

In [ ]:
CLDRIVE_TABLE_NAMES = ["CLSmith w. cldrive", "GitHub", "CLgen"]
CLDRIVE_TABLES = [cldriveCLSmithResult, GitHubResult, CLgenResult]

q = session.query(cldriveParams).order_by(
        cldriveParams.size,
        cldriveParams.gsize_x, cldriveParams.gsize_y, cldriveParams.gsize_z,
        cldriveParams.lsize_x, cldriveParams.lsize_y, cldriveParams.lsize_z,
        cldriveParams.generator, cldriveParams.scalar_val, cldriveParams.optimizations)

# push LaTex to Overleaf
!cd ~/docs/paper-project_b/ && git pull --rebase
data = []
for param in q:
    data.append([param.size, param.gsize, param.lsize, param.optimizations_on_off])
table = pd.DataFrame(data, index=range(1, len(data)+1), columns=[
    "Dataset Size", "Global size", "Workgroup size", "Optimizations"])
with open(os.path.expanduser("~/docs/paper-project_b/build/tab/cldrive-params.tex"), "w") as outfile:
    table.to_latex(buf=outfile)
!cd ~/docs/paper-project_b/build && git add . && git commit -m "auto: build/tab/cldrive-params.tex" && git push
table

# Experimental Results

## Runtimes

Excluding runs which terminated in non-zero status:

In [ ]:
import numpy as np

runtimes = [np.array(session.query(table.runtime).filter(table.status == 0).all()) for table in TABLES]
data = [
    ("Min", [r.min() for r in runtimes]),
    ("Median", [np.median(r) for r in runtimes]),
    ("Mean", [r.mean() for r in runtimes]),
    ("Max", [r.max() for r in runtimes])
]
i, d = zip(*data)
runtimes = pd.DataFrame(list(d), index=i, columns=TABLE_NAMES)
runtimes

## Outcomes & Classifications

**Pandas tables of outcomes**

In [ ]:
outcomes = {}

for name, table in zip(CL_LAUNCHER_TABLE_NAMES + CLDRIVE_TABLE_NAMES, CL_LAUNCHER_TABLES + CLDRIVE_TABLES):
    r = []
    for testbed in session.query(Testbed).all():
        nresult = session.query(table).filter(table.testbed == testbed).count()

        q = session.query(table.outcome, sql.func.count(table.outcome)).filter(
            table.testbed == testbed).group_by(table.outcome).order_by(
                sql.desc(sql.func.count(table.outcome)))

        for outcome, count in q.all():
            ratio = (count / nresult) * 100
            r.append((DEVICES.get(testbed.device, testbed.device), outcome, count, ratio))
    outcomes[name] = pd.DataFrame(r, columns=["Device", "Outcome", "Count", "% of Total Results"])

print("done.")

**Pandas tables of classifications**

In [ ]:
classifications = {}

classificationsSort = [
    'Invalid testcase',
    'Build failure',
    'Runtime crash',
    'No majority',
    'Wrong code',
    'Okay'
]

def escape(val):
    if val is None:
        return val
    else:
        return str(classificationsSort.index(val)) + ". " + val

for name, table in zip(CL_LAUNCHER_TABLE_NAMES + CLDRIVE_TABLE_NAMES, CL_LAUNCHER_TABLES + CLDRIVE_TABLES):
    r = []
    for testbed in session.query(Testbed).all():
        nresult = session.query(table).filter(table.testbed == testbed).count()

        q = session.query(table.classification, sql.func.count(table.classification)).filter(
            table.testbed == testbed).group_by(table.classification).order_by(
                sql.desc(sql.func.count(table.classification)))

        for val, count in q.all():
            ratio = (count / nresult) * 100
            r.append((DEVICES.get(testbed.device, testbed.device), escape(val), count, ratio))
    classifications[name] = pd.DataFrame(r, columns=["Device", "Classification", "Count", "% of Total Results"])

print("done.")

In [ ]:
classifications["CLgen w. cl_launcher"]

## Experimental Results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from labm8 import viz
%matplotlib inline

def plot_outcomes(table, name, dictname=outcomes, key='Outcome'):
    ax = dictname[name].pivot('Device', key)['Count'].plot(
        kind='bar', stacked=True, colormap="Reds_r", sort_columns=True)

    nprog = session.query(table.program_id).group_by(table.program_id).count()
    nparam = session.query(table.params_id).group_by(table.params_id).count()
    plt.title(f"{nprog} {name} x {nparam} parameters")
    plt.ylabel("Results")
    plt.xlabel("")

    plt.ylim(0, nprog * nparam)

    # reverse legend order (because plot stacks from bottom to top, and legend goes from top to bottom)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles[::-1], labels[::-1], loc='center left', bbox_to_anchor=(1, 0.5))

    viz.finalise(figsize=(3.5, 8))
    
    
def summarize(table_name):
    """ summarize a table of classifications """
    table = classifications[table_name]

    def get_val(classification):
        try:
            return table.loc[
                (table['Device'] == device) & (table['Classification'] == classification)]['Count'].values[0]
        except IndexError:
            return 0
    
    columns = ['Platform', 'Device', 'Driver', 'Invalid Testcases', 'Build Failures', 'Runtime Crashes', 'Incorrect Outputs', 'Okay']
    devices = sorted(set(table['Device'].values))

    d = []    
    for device in devices:
        lookup = dict((v, k) for k, v in DEVICES.items())
        full_name = lookup.get(device, device)
        
        # lookup the testbed
        q = session.query(Testbed).filter(Testbed.device == full_name).all()
        if len(q) != 1:
            raise q
        testbed = q[0]
        
        r = [
            PLATFORMS.get(testbed.platform, testbed.platform),
            device,
            DRIVERS.get(testbed.driver, testbed.driver),
            get_val('0. Invalid testcase'),
            get_val('1. Build failure'),
            get_val('2. Runtime crash'),
            get_val('3. Wrong code'),
            get_val('4. Okay'),
        ]
        d.append(r)
    summary = pd.DataFrame(d, columns=columns, index=range(1, len(devices)+1))

    !cd ~/docs/paper-project_b/ && git pull --rebase >/dev/null
    name = '-'.join(table_name.split())
    with open(os.path.expanduser(f"~/docs/paper-project_b/build/tab/results-{name}.tex"), "w") as outfile:
        summary.to_latex(buf=outfile)
    !cd ~/docs/paper-project_b/build && git add . && git commit -m "auto: summarize table" >/dev/null && git push >/dev/null
    return summary

### CLSmith

In [ ]:
outcomes["CLSmith"]

In [ ]:
summarize('CLSmith')

In [ ]:
import numpy as np

def lc(src):
    return len(src.strip().split("\n"))

lcs = np.array([lc(row[0]) for row in session.query(CLSmithProgram.src)])

In [ ]:
np.median(lcs)

In [ ]:
plot_outcomes(CLSmithResult, "CLSmith", dictname=classifications, key='Classification')

### CLgen w. cl_launcher

In [ ]:
outcomes["CLgen w. cl_launcher"]

In [ ]:
summarize('CLgen w. cl_launcher')

In [ ]:
plot_outcomes(cl_launcherCLgenResult, "CLgen w. cl_launcher", dictname=classifications, key='Classification')

### CLSmith w. cldrive

In [ ]:
outcomes["CLSmith w. cldrive"]

In [ ]:
summarize('CLSmith w. cldrive')

In [ ]:
plot_outcomes(cldriveCLSmithResult, "CLSmith w. cldrive", dictname=classifications, key='Classification')

### GitHub

In [ ]:
outcomes["GitHub"]

In [ ]:
summarize('GitHub')

In [ ]:
plot_outcomes(GitHubResult, "GitHub", dictname=classifications, key='Classification')

### CLgen

In [ ]:
outcomes["CLgen"]

In [ ]:
summarize('CLgen')

In [ ]:
plot_outcomes(CLgenResult, "CLgen", dictname=classifications, key='Classification')